# LLM for query optimization

The problem:
- Singularity 6 / Palia: We should search Reddit for Palia
- Akili Interactive / EndeavorRx: We should search Reddit for EndeavorRx
- Rad.ai Omni: We should search Reddit for Rad.ai Omni



In [46]:
from core import Seed, init

init()

In [54]:
seed = Seed.init("Akili Interactive", "EndeavorRx")

queries = [
    # f'site:reddit.com "{seed.company}" "{seed.product}"',
    # f'site:reddit.com "{seed.product}"',
    f'"{seed.company}"',
    f'"{seed.company}" news',
]

queries

['"Akili Interactive"', '"Akili Interactive" news']

In [55]:
from google_search import search

queries_results = [list(search(query, num=40)) for query in queries]
queries_results

2024-09-19 08:24:35.191 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - "Akili Interactive"', 'totalResults': '25800', 'searchTerms': '"Akili Interactive"', 'count': 10, 'startIndex': 1, 

[[SearchResult(title='Akili Interactive', link='https://www.akiliinteractive.com/', snippet='Akili Interactive is a prescription digital medicine company combining scientific and clinical rigor with the ingenuity of the tech industry to reinvent\xa0...', formattedUrl='https://www.akiliinteractive.com/'),
  SearchResult(title='ADHD game developer Akili Interactive to be taken private, after $34 ...', link='https://www.fiercebiotech.com/medtech/adhd-game-developer-akili-interactive-be-taken-private-after-being-sold-34m', snippet='May 30, 2024 ... ... Akili Interactive has found a willing buyer. Virtual Therapeutics, the maker of virtual reality software for mental health and wellness\xa0...', formattedUrl='https://www.fiercebiotech.com/.../adhd-game-developer-akili-interactive-be...'),
  SearchResult(title='Products — Akili Interactive', link='https://www.akiliinteractive.com/products', snippet='Akili Interactive. Science & Tech Products Blog Join Us News Contact. Back Overview Product D

In [92]:
from typing import Dict, List
from google_search import SearchResult

def index_search_results(search_results: List[SearchResult]) -> Dict[int, SearchResult]:
    indexed_results = {}

    # TODO: Deduplicate search results

    for i, search_result in enumerate(search_results):
        indexed_results[i+1] = search_result

    return indexed_results

def format_search_results(indexed_search_results: Dict[int, SearchResult]) -> str:
    result = ""

    for i, search_result in sorted(indexed_search_results.items()):
        domain = search_result.link.split('/')[2]
        result += f"{i}. {search_result.title} (from {domain})\n"
        result += f"{search_result.snippet}\n\n"

    return result

print(format_search_results(index_search_results(queries_results[0][:5])))

1. Akili Interactive (from www.akiliinteractive.com)
Akili Interactive is a prescription digital medicine company combining scientific and clinical rigor with the ingenuity of the tech industry to reinvent ...

2. ADHD game developer Akili Interactive to be taken private, after $34 ... (from www.fiercebiotech.com)
May 30, 2024 ... ... Akili Interactive has found a willing buyer. Virtual Therapeutics, the maker of virtual reality software for mental health and wellness ...

3. Products — Akili Interactive (from www.akiliinteractive.com)
Akili Interactive. Science & Tech Products Blog Join Us News Contact. Back Overview Product Development COVID Brain Fog 2023 ADHD ...

4. Akili Interactive, maker of digital therapeutics, announces layoffs (from www.statnews.com)
Apr 30, 2024 ... a phone screen displays Akili Interactive's logo – health tech coverage from STAT Adobe ... Founded in 2011, Akili Interactive made waves in 2020 ...

5. Join Us — Akili Interactive (from www.akiliinteractive.co

In [93]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field
from typing import List

indexed_search_results = index_search_results(queries_results[0] + queries_results[1])
formatted_search_results = format_search_results(indexed_search_results)

class RatedResult(BaseModel):
    result_number: int = Field(description="Number of the search result", ge=min(indexed_search_results.keys()), le=max(indexed_search_results.keys()))

    product_relevance: int = Field(description="Relevance of the search result for product feedback and critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)
    company_relevance: int = Field(description="Relevance of the search result to the company critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)

class RatedResults(BaseModel):
    rated_queries: List[RatedResult] = Field(description="List of relevance-rated search results")

_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Review a list of search results and rate their relevance to A) the company {company} and B) the product {product} made by {company} on a 1-5 scale.
            """,
        ),
        (
            "human",
            """
Search results:
{search_results}
            """,
        ),
    ]
)


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
runnable = _prompt | llm.with_structured_output(RatedResults)
llm_result = runnable.with_config({"run_name": "POC: Rerank search results"}).invoke(
    {
        "company": seed.company,
        "product": seed.product,
        "search_results": formatted_search_results,
    }
)

llm_result

RatedResults(rated_queries=[RatedResult(result_number=1, product_relevance=5, company_relevance=5), RatedResult(result_number=2, product_relevance=3, company_relevance=5), RatedResult(result_number=3, product_relevance=5, company_relevance=5), RatedResult(result_number=4, product_relevance=3, company_relevance=5), RatedResult(result_number=5, product_relevance=1, company_relevance=2), RatedResult(result_number=6, product_relevance=3, company_relevance=5), RatedResult(result_number=7, product_relevance=5, company_relevance=5), RatedResult(result_number=8, product_relevance=4, company_relevance=5), RatedResult(result_number=9, product_relevance=3, company_relevance=5), RatedResult(result_number=10, product_relevance=3, company_relevance=4), RatedResult(result_number=11, product_relevance=4, company_relevance=4), RatedResult(result_number=12, product_relevance=2, company_relevance=3), RatedResult(result_number=13, product_relevance=1, company_relevance=2), RatedResult(result_number=14, pr

In [94]:
print(formatted_search_results)

1. Akili Interactive (from www.akiliinteractive.com)
Akili Interactive is a prescription digital medicine company combining scientific and clinical rigor with the ingenuity of the tech industry to reinvent ...

2. ADHD game developer Akili Interactive to be taken private, after $34 ... (from www.fiercebiotech.com)
May 30, 2024 ... ... Akili Interactive has found a willing buyer. Virtual Therapeutics, the maker of virtual reality software for mental health and wellness ...

3. Products — Akili Interactive (from www.akiliinteractive.com)
Akili Interactive. Science & Tech Products Blog Join Us News Contact. Back Overview Product Development COVID Brain Fog 2023 ADHD ...

4. Akili Interactive, maker of digital therapeutics, announces layoffs (from www.statnews.com)
Apr 30, 2024 ... a phone screen displays Akili Interactive's logo – health tech coverage from STAT Adobe ... Founded in 2011, Akili Interactive made waves in 2020 ...

5. Join Us — Akili Interactive (from www.akiliinteractive.co

In [95]:
import numpy as np

print(f"""
Number of search results: {len(llm_result.rated_queries)}
Average product relevance: {np.mean([rated_result.product_relevance for rated_result in llm_result.rated_queries]):.1f}
Average company relevance: {np.mean([rated_result.company_relevance for rated_result in llm_result.rated_queries]):.1f}
""")


Number of search results: 80
Average product relevance: 2.8
Average company relevance: 3.9



In [96]:
link2query = {}

for query, query_results in zip(queries, queries_results):
    for query_result in query_results:
        link2query[query_result.link] = query

len(link2query)

65

In [97]:
print("""# Product-relevant search results\n\n""")

for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.product_relevance, reverse=True)[:10]:
    search_result = indexed_search_results[rated_result.result_number]
    print(f"""
## [{search_result.title}]({search_result.link})
{search_result.snippet}

Product relevance: {rated_result.product_relevance}
Company relevance: {rated_result.company_relevance}

""")

print("""# Company-relevant search results\n\n""")
for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.company_relevance, reverse=True)[:10]:
    search_result = indexed_search_results[rated_result.result_number]
    print(f"""
## [{search_result.title}]({search_result.link})
{search_result.snippet}

Product relevance: {rated_result.product_relevance}
Company relevance: {rated_result.company_relevance}

""")


# Product-relevant search results



## [Akili Interactive](https://www.akiliinteractive.com/)
Akili Interactive is a prescription digital medicine company combining scientific and clinical rigor with the ingenuity of the tech industry to reinvent ...

Product relevance: 5
Company relevance: 5



## [Products — Akili Interactive](https://www.akiliinteractive.com/products)
Akili Interactive. Science & Tech Products Blog Join Us News Contact. Back Overview Product Development COVID Brain Fog 2023 ADHD ...

Product relevance: 5
Company relevance: 5



## [EndeavorRx® - ADHD Video Game Treatment for Kids | FDA ...](https://www.endeavorrx.com/)
... Akili Interactive Labs, Inc. Other trademarks are trademarks or registered trademarks of their respective owners. Veeva #US-00405 04/22. ×. How do I find my ...

Product relevance: 5
Company relevance: 5



## [The story behind an FDA-Approved Video Game Treatment for ADHD](https://themedicinemaker.com/discovery-development/the-story-behind-an-fd